<a href="https://colab.research.google.com/github/putaoo/C-OCR/blob/master/Notebooks/finetune/xtts/colab_xtts_finetune_webui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Welcome to the *xtts*-finetune-webui gradio gui!

This webui is a slightly modified copy of the official webui for finetune xtts.

If you are looking for an option for normal XTTS use look here https://github.com/daswer123/xtts-webui

In [ ]:
# @title 🛠️ Install requirments
#!DEBIAN_FRONTEND=noninteractive
!sudo apt-get update # && sudo apt-get -y upgrade
!sudo apt-get -y install libegl1
!sudo apt-get -y install libopengl0
!sudo apt-get -y install libxcb-cursor0
!pip install -r https://raw.githubusercontent.com/daswer123/xtts-finetune-webui/main/requirements.txt
!pip install gradio==4.44.1
!pip install fastapi==0.103.1
!pip install pydantic==2.3.0

#More requirements
!nvcc --version #check cuda version if things break
!pip uninstall torch torchvision torchaudio -y
!pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu121


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Fetched 257 kB in 1s (269 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package list

In [ ]:
# @title 🚀 Run interface
%cd /content/
!git clone https://github.com/DrewThomasson/xtts-finetune-webui.git
%cd /content/xtts-finetune-webui
!python xtts_demo.py --share

In [ ]:
import shutil
import requests
import os
from tqdm import tqdm  # Progress bar library

# Define the paths
finetune_dir = '/content/xtts-finetune-webui/finetune_models/ready'  # @param {type:"string"}
dataset_dir = '/content/xtts-finetune-webui/finetune_models/dataset'  # @param {type:"string"}

# Create a temporary directory to collect both folders before zipping
temp_dir = "/content/temp_finetune_dataset"
os.makedirs(temp_dir, exist_ok=True)

# Copy both directories into the temporary directory with a progress bar
def copy_with_progress(src, dst):
    total_files = sum(len(files) for _, _, files in os.walk(src))
    with tqdm(total=total_files, desc=f"Copying {os.path.basename(src)}") as pbar:
        for root, _, files in os.walk(src):
            rel_path = os.path.relpath(root, src)
            target_path = os.path.join(dst, rel_path)
            os.makedirs(target_path, exist_ok=True)
            for file in files:
                shutil.copy(os.path.join(root, file), target_path)
                pbar.update(1)

copy_with_progress(finetune_dir, os.path.join(temp_dir, "ready"))
copy_with_progress(dataset_dir, os.path.join(temp_dir, "dataset"))

# Create a zip file of the combined directories with progress
zip_filename = "finetune_and_dataset.zip"
with tqdm(total=100, desc="Zipping files") as pbar:
    shutil.make_archive("finetune_and_dataset", 'zip', root_dir=temp_dir)
    pbar.update(100)

# Define a function to stream the upload with a progress bar
def upload_with_progress(file_path, url):
    file_size = os.path.getsize(file_path)
    with open(file_path, 'rb') as f, tqdm(
        total=file_size, unit='B', unit_scale=True, desc="Uploading"
    ) as progress:
        response = requests.post(
            url,
            files={"file": (file_path, f)},
            stream=True,
            headers={"Connection": "keep-alive"},
        )
        # Update the progress bar as chunks are sent
        for chunk in response.iter_content(chunk_size=4096):
            if chunk:
                progress.update(len(chunk))
    return response

# Upload the zip file to file.io with a progress bar
response = upload_with_progress(zip_filename, "https://file.io/?expires=1d")

# Parse the response and display the download link
if response.status_code == 200:
    download_link = response.json().get('link', 'Error: No link found.')
    print(f"Your file is ready: {download_link}")
else:
    print(f"Failed to upload: {response.status_code} - {response.text}")
